### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 5 sobre las plantas medicinales creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga una página principal que mostrará un conjunto enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * __Servicio 1__: Mostrar información sobre una planta seleccionada. Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que dispondrá varios desplegables donde podrá seleccionar la planta. Un seleccionable para elegir el grupo , y otro para seleccionar la planta del grupo que ha elegido.  Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página que mostrará la descripción de la planta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]
   
   
   * __Servicio 2__:Buscar planta por palabra clave. Si elige esta opción se le pedirá al usuario que introduzca un conjunto de palabras que se utilizarán para realizar una búsqueda sobre el texto de las descripciones asociadas a las plantas que aparecen en cada página. A continuación, si ha introducido más de una palabra, se le preguntará si quiere realizar una búsqueda de tipo "AND" o una búsqueda de tipo "OR", es decir si busca plantas donde aparece todas las palabras introducidas o si busca plantas donde aparecen alguna de las palabras introducidas. Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página con un listado de todas las plantas junto a las descripciones de las mismas que cumplen las condiciones de búsqueda.  En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]

   * __Servicio 3__:Buscar plantas por enfermedades. Si elige esta opción se le mostrará un listado chequeable donde seleccionará una o más enfermedades. Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página con todas las plantas que pueden venir bien para esa enfermedad. Para ello se proporciona junto a la práctica un csv con nombres de enfermedades que servirá de entrada para generar el listado de enfermedades.En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]

## Normas de entrega

* Fecha tope de entrega: 31/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [ ]:
# Para el desarrollo Web
from bottle import run, route, error, request, template, static_file;

# Para el Web Scrapping
import re;
import requests;
from bs4 import BeautifulSoup;

#**********************************************************************************************************************

class Model:
    
    __instance = None;
    
    def __init__(self):
        
        self.__gruposPlantas = {};
        self.__mapaDescripciones = {};
        self.__webs = [
            'https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(A-B)',
            'https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(C)',
            'https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(D-G)',
            'https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(H-M)',
            'https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(N-Z)',
        ];
    
    def getGruposPlantas(self): return self.__gruposPlantas;
    def getMapaDescripciones(self): return self.__mapaDescripciones;
    
    @staticmethod
    def getInstance():
        
        if(Model.__instance == None):
            
            Model.__instance = Model();
        
        return Model.__instance;
    
    def obtainDataUsingWebScrapping(self):
    
        self.__recolectarDatos();
    
    def obtainPlantsForDiseases(self,diseases):
        
        nombresCoincidencias = []
    
        for disease in diseases:
            
            print("ENFERMEDAD SOLICITADA:",disease)
            nombresCoincidencias.append(self.__buscarCoincidencias(disease))

        nombresResultados = self.__union(nombresCoincidencias)
    
        data = dict();
    
        print()
        print("Se han encontrado", len(nombresResultados), "coincidencias: ")
        for nombre in nombresResultados:
        
            data[nombre] = self.__mapaDescripciones[nombre];
            
        return data;
    
    def __recolectarDatos(self):
        i = 1
        for web in self.__webs:
            # print("Recolectando datos de plantas... (", i, "/", len(self.webs), ")")
            plantas = self.__extraerPlantas(web)
            self.__guardarPlantas(plantas, i)
            i += 1

    def __guardarPlantas(self,plantas, grupo):
        self.__gruposPlantas[grupo] = {}
        i = 1
        for planta in plantas:
            self.__gruposPlantas[grupo][i] = planta
            # print(self.gruposPlantas[grupo][i]);
            i += 1

    def __buscarCoincidencias(self,palabra):
        coincidencias = []
        regexp = re.compile(r"\b" + palabra + r"\b", re.IGNORECASE)
        for planta, descripcion in self.__mapaDescripciones.items():
            if regexp.search(descripcion):
                coincidencias.append(planta)
        return coincidencias

    def __union(self,listas):
        resultado = set()
        for lista in listas:
            resultado = resultado.union(lista);
        return resultado

    def __mostrarResultados(self,resultados):
        print()
        print("Se han encontrado", len(resultados), "coincidencias: ")
        for nombre in resultados:
            print('Planta:', nombre)
            print('Descripción:', mapaDescripciones[nombre])
            print()
        
    def __extraerPlantas(self,web):
        req = requests.get(web)
        soup = BeautifulSoup(req.text, "html.parser")
        [tag.extract() for tag in soup.select('.reference')]
        itemsLista = soup.select('#toc > ul li > a')
        plantas = []
        for item in itemsLista:
            idPlanta = item['href'].replace('#', '')
            tituloPlanta = soup.find(id=idPlanta)
            if tituloPlanta.parent.name != 'h3':
                continue

            infoPlanta = tituloPlanta.parent.find_next_sibling('dl')
            if infoPlanta is None:
                continue

            nombreCientifico = infoPlanta.select_one('p a,dt a')
            if nombreCientifico is None:
                nombreCientifico = 'Desconocido'
            else:
                nombreCientifico = nombreCientifico.get_text().strip()

            bloquesInfo = infoPlanta.findChildren()

            if infoPlanta.name == 'p':
                descripcion = infoPlanta.find_next_sibling().get_text().strip()
            elif len(bloquesInfo) > 0:
                bloquesInfo[0].extract()
                descripcion = infoPlanta.get_text().strip()

            plantas.append({
                "nombre": tituloPlanta.get_text(),
                "nombre_cientifico": nombreCientifico,
                "descripcion": descripcion
            })
            
            self.__mapaDescripciones[tituloPlanta.get_text()] = descripcion

        return plantas
  
#**********************************************************************************************************************

class DispatcherView:
    
    __instance = None;
    
    @staticmethod
    def getInstance():
        
        if(DispatcherView.__instance == None):
            
            DispatcherView.__instance = DispatcherView();
        
        return DispatcherView.__instance;

    def generateMainMenu(self):
        
        return static_file('Main.html', root='');
    
    def generateService(self,data):
        
        return template('Service.tpl', dict=data);
                        
    def generateResultsForService(self,data):
        
        return template('Results.tpl', dict=data);
    
#**********************************************************************************************************************

class Controller:
    
    __instance = None;
    
    @staticmethod
    def getInstance():
        
        if(Controller.__instance == None):
            
            Controller.__instance = Controller();
        
        return Controller.__instance;
    
    def action(self,context):
        
        if(context["event"] == "INITALIZE"):
        
            DispatcherView.getInstance();
        
            Model.getInstance().obtainDataUsingWebScrapping();
            
        elif(context["event"] == "SHOW_MAIN_MENU_TO_USER"):
            
            return DispatcherView.getInstance().generateMainMenu();
        
        elif(context["event"] == "PROVIDE_SERVICE_1_TO_USER"):
            
            context["object"]["groupsOfPlants"] = Model.getInstance().getGruposPlantas(); 
            
            return DispatcherView.getInstance().generateService(context["object"]);

        elif(context["event"] == "PROVIDE_SERVICE_1_BIS_TO_USER"):
        
            selectedGroupIndex = context["object"]["selectedGroupIndex"];
        
            context["object"]["selectedGroup"] = Model.getInstance().getGruposPlantas()[selectedGroupIndex];
        
            return DispatcherView.getInstance().generateService(context["object"]);
        
        elif(context["event"] == "PROVIDE_SERVICE_2_TO_USER"):
            
            return DispatcherView.getInstance().generateService(context["object"]);
        
        elif(context["event"] == "PROVIDE_SERVICE_3_TO_USER"):
            
            return DispatcherView.getInstance().generateService(context["object"]);
        
        elif(context["event"] == "SHOW_SERVICE_3_RESULTS_TO_USER"):
            
            context["object"]["plantsForDiseases"] = Model.getInstance().obtainPlantsForDiseases(context["object"]["diseases"]);
            
            return DispatcherView.getInstance().generateResultsForService(context["object"]);
        
#**********************************************************************************************************************

@route('/')
@route('/MainMenu')
def main():
    
    return Controller.getInstance().action({ "event": "SHOW_MAIN_MENU_TO_USER" , "object": None });

@route('/Servicio1')
def service1():
    
    return Controller.getInstance().action(
        { 
            "event": "PROVIDE_SERVICE_1_TO_USER", 
            "object": 
            {
                "serviceName": "SERVICIO 1"
            } 
        }
    );

@route('/Servicio1Bis', method='POST')
def service1Bis():
    
    return Controller.getInstance().action(
        { 
            "event": "PROVIDE_SERVICE_1_BIS_TO_USER", 
            "object": 
            {
                "serviceName": "SERVICIO 1 BIS",
                "selectedGroupIndex": int(request.forms.get("grupo"))
            } 
        }
    );


@route('/Service1Results', method='POST')
def service1Results():
    
    return Controller.getInstance().action(
        { 
            "event": "SHOW_SERVICE_1_RESULTS_TO_USER", 
            "object": 
            {
                "serviceName": "SERVICIO 1"
            } 
        }
    );

@route('/Servicio2')
def service2():
    
    return Controller.getInstance().action(
        { 
            "event": "PROVIDE_SERVICE_2_TO_USER", 
            "object": 
            {
                "serviceName": "SERVICIO 2"
            } 
        }
    );

@route('/Servicio3')
def service3():
    
    return Controller.getInstance().action(
        { 
            "event": "PROVIDE_SERVICE_3_TO_USER", 
            "object": 
            {
                "serviceName": "SERVICIO 3"
            } 
        }
    );

@route('/Service3Results', method='POST')
def service3Results():
    
    return Controller.getInstance().action(
        { 
            "event": "SHOW_SERVICE_3_RESULTS_TO_USER" , 
            "object":
            {
                "serviceName": "SERVICIO 3",
                "diseases": ["".join(inputName) for inputName in request.forms]
            } 
        }
    );


@route('/Styles.css')
@route('/Portada.jpg')
def getResource():
    
    requestedResource = request.route.rule;
    
    if(requestedResource == "/Styles.css"): return static_file('Styles.css',root='');
    elif(requestedResource == "/Portada.jpg"): return static_file('Portada.jpg',root='')
    else: return None; 

@error(404)
def error404():
    
    return "LA PAGINA SOLICITADA NO HA SIDO ENCONTRADA";

#**********************************************************************************************************************

Controller.getInstance().action({ "event": "INITALIZE" , "object": None });

run(host='localhost', port=8080,debug=True)

Bottle v0.13-dev server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [29/Oct/2019 12:46:19] "GET / HTTP/1.1" 200 530
127.0.0.1 - - [29/Oct/2019 12:46:20] "GET /Portada.jpg HTTP/1.1" 200 142498
